In [1]:
import cv2
import numpy as np
import math

In [2]:
#our region of interest is trapezium
#it takes a image which has SINGLE value at (x,y) location. for ex grayscale image
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    
    # Retrieve the number of color channels of the image.
    #channel_count = img.shape[2]
    
    #check if image is grayscale or color
    shape = img.shape
    
    # color used to fill polygon
    if(len(shape) == 3):               #its a color image
        mask_color = (255,)*shape[-1]
    else:                              #its a gray image or single channel
        mask_color = 255
      
    # Fill the polygon with white
    cv2.fillPoly(mask, vertices, mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [3]:
def hsv_filter(image, min_val_y, max_val_y,  min_val_w, max_val_w):
    """
    A function returning a mask for pixels within min_val - max_val range
    Inputs:
    - image - a BGR image you want to apply function on
    - min_val_y - array of shape (3,) giving minumum HSV values for yellow color
    - max_val_y - array of shape (3,) giving maximum HSV values for yellow color
    - min_val_w - array of shape (3,) giving minumum HSV values for white color
    - max_val_w - array of shape (3,) giving maximum HSV values for white color
    Returns:
    - img_filtered - image of pixels being in given threshold
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask_yellow = cv2.inRange(hsv, min_val_y, max_val_y)
    mask_white = cv2.inRange(hsv, min_val_w, max_val_w)
    mask = cv2.bitwise_or(mask_yellow, mask_white)
    img_filtered = cv2.bitwise_and(image, image, mask=mask)
    
    return img_filtered

In [4]:
def draw_lines(img, lines, color, thickness):
    # If there are no lines to draw, exit.
    if lines is None:
        return
    
    # Make a copy of the original image.
    img = np.copy(img)
    
    # Create a blank image that matches the original in size.
    line_img = np.zeros(
        (
            img.shape[0],
            img.shape[1],
            3            #remove this if its a grayscale image
        ),
        dtype=np.uint8,
    )
    
    # Loop over all lines and draw them on the blank image.
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
    
    # Merge the image with the lines onto the original.
    img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    
    # Return the modified image.
    return img

In [5]:
#Applying canny edge and thresholding using median of pixels
def autoCanny(img, sigma=0.33):
    img = img[:,:,2]
    #get median of gray scale image
    med = np.median(img)
    
    #compute lower and upper threshold
    lower = int(max(0, (1-sigma)*med))
    upper = int(min(255, (1+sigma)*med))
    edgedImg = cv2.Canny(img, lower, upper)
    
    #return image with prominent edges
    return edgedImg

In [6]:
def gamma_correction(RGBimage, correct_param = 0.35,equalizeHist = False):
    red = RGBimage[:,:,2]
    green = RGBimage[:,:,1]
    blue = RGBimage[:,:,0]
    
    red = red/255.0
    red = cv2.pow(red, correct_param)
    red = np.uint8(red*255)
    if equalizeHist:
        red = cv2.equalizeHist(red)
    
    green = green/255.0
    green = cv2.pow(green, correct_param)
    green = np.uint8(green*255)
    if equalizeHist:
        green = cv2.equalizeHist(green)
        
    
    blue = blue/255.0
    blue = cv2.pow(blue, correct_param)
    blue = np.uint8(blue*255)
    if equalizeHist:
        blue = cv2.equalizeHist(blue)
    

    output = cv2.merge((blue,green,red))
    return output

In [8]:
cap = cv2.VideoCapture('../dataset/night-lane.mp4')

#get height and width of frame
ret, frame = cap.read()
height, width, channel = frame.shape

#keeping the portion of image which contains lane
topLeftPt = (width*(3.0/8), height*(3/5))
topRightPt = (width*(5.0/8), height*(3/5))

#polygon has 6 vertices
region_of_interest_points = [
(0, height),
(0, height*(3.4/5)),
topLeftPt,
topRightPt,
(width, height*(3.4/5)),
(width, height),
]

#defining color thresholds
min_val_y = np.array([15,80,190])
max_val_y = np.array([30,255,255])
min_val_w = np.array([0,0,195])
max_val_w = np.array([255, 80, 255])

while True:

    # Take each frame
    ret, frame = cap.read()
    
    gamma = gamma_correction(frame, correct_param = 0.2,equalizeHist = False)
    #cv2.imshow('gamma', gamma)

    cropped = region_of_interest(gamma, np.array([region_of_interest_points], np.int32))
    #cv2.imshow('cropped', cropped)

    bilateral = cv2.bilateralFilter(cropped, 9, 80, 80)
    #cv2.imshow('bilateral', bilateral)

    hsv = hsv_filter(bilateral, min_val_y, max_val_y,  min_val_w, max_val_w)
    #cv2.imshow('hsv', hsv)
    
    #perform canny edge on gray scale image
    cannyImg = autoCanny(hsv, 0.33)
    
    #cv2.imshow('cannyImg', cannyImg)
    
    #crop image
    #croppedImg = region_of_interest(cannyImg, np.array([region_of_interest_points], np.int32))
    #cv2.imshow('Region of Interest',croppedImg)
    
    #Hough transform
    lines = cv2.HoughLinesP(cannyImg, rho=6, theta=np.pi / 60,threshold=200, lines=np.array([]),minLineLength=40,maxLineGap=25)
    
    
    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = (y2 - y1) / (x2 - x1) # <-- Calculating the slope.
            if math.fabs(slope) < 0.4: # <-- Only consider extreme slope
                continue
            if slope <= 0: # If the slope is negative, left group.
                left_line_x.extend([x1, x2])
                left_line_y.extend([y1, y2])
            else: # Otherwise, right group.
                right_line_x.extend([x1, x2])
                right_line_y.extend([y1, y2])
                
    min_y = int(frame.shape[0] * (5 / 7))       # Just below the horizon
    max_y = frame.shape[0]                 # The bottom of the image
    
    #if image don't have left lane or right lane then don't draw line
    if len(left_line_x) & len(left_line_y) & len(right_line_x) & len(right_line_y):
        #get x co-ordinates of left line
        poly_left = np.poly1d(np.polyfit(
            left_line_y,
            left_line_x,
            deg=1
        ))
        left_x_start = int(poly_left(max_y))
        left_x_end = int(poly_left(min_y))

        #get x co-ordinate of right line
        poly_right = np.poly1d(np.polyfit(
            right_line_y,
            right_line_x,
            deg=1
        ))
        right_x_start = int(poly_right(max_y))
        right_x_end = int(poly_right(min_y))

        #add original image and line
        imageWithLine = draw_lines(frame,
            [[
                [left_x_start, max_y, left_x_end, min_y],
                [right_x_start, max_y, right_x_end, min_y],
            ]],
            [0,255,0],
            thickness=5)

        #display image with line over it
        cv2.imshow('imageWithLine', imageWithLine)
    
    k = cv2.waitKey(25) & 0xFF
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not iterable